# IMPORT REQUIRED LIBRARIES

In [1]:
import os, sys, random, shutil
import xml.etree.ElementTree as ET
from glob import glob
import pandas as pd
from shutil import copyfile
import pandas as pd
from sklearn import preprocessing, model_selection
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import patches
import numpy as np

In [2]:
annotations = sorted(glob(r'C:\Users\saikr\object_detection_p2\obj_dataset\Annotations\*.xml'))

df = []
cnt = 0
for file in annotations:
    prev_filename = file.split('\\')[-1].split('.x')[0] + '.jpg'
    filename = str(cnt) + '.jpg'
    row = []
    parsedXML = ET.parse(file)
    for node in parsedXML.getroot().iter('object'):
        uniform_type = node.find('name').text
        xmin = int(node.find('bndbox/xmin').text)
        xmax = int(node.find('bndbox/xmax').text)
        ymin = int(node.find('bndbox/ymin').text)
        ymax = int(node.find('bndbox/ymax').text)
        row = [prev_filename, filename, uniform_type, xmin, xmax, ymin, ymax]
        df.append(row)
    cnt += 1

data = pd.DataFrame(df, columns=['prev_filename', 'filename', 'uniform_type', 'xmin', 'xmax', 'ymin', 'ymax'])
data['uniform_type']= data['uniform_type'].replace('Traffic polic','Traffic police')

data[['prev_filename','filename', 'uniform_type', 'xmin', 'xmax', 'ymin', 'ymax']].to_csv(r'C:\Users\saikr\object_detection_p2\uniform_data1_detection.csv', index=False)
data.head(10)

,prev_filename,filename,uniform_type,xmin,xmax,ymin,ymax
0,OMG-Oh-My-God-court-room-drama-e1474114545186.jpg,0.jpg,lawyer,26,221,108,387
1,OMG-Oh-My-God-court-room-drama-e1474114545186.jpg,0.jpg,lawyer,456,700,107,319
2,Telangana-Traffic-police.jpg,1.jpg,Traffic police,422,654,177,800
3,Telangana-Traffic-police.jpg,1.jpg,Traffic police,754,956,147,461
4,ap1.jpg,2.jpg,military force,199,325,112,394
5,ap10.jpg,3.jpg,military force,15,272,211,336
6,ap10.jpg,3.jpg,military force,397,612,215,332
7,ap100.jpg,4.jpg,military force,81,173,179,408
8,ap100.jpg,4.jpg,military force,395,476,220,350
9,ap101.jpg,5.jpg,military force,12,92,80,303


In [3]:
data.shape

(2231, 7)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2231 entries, 0 to 2230
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   prev_filename  2231 non-null   object
 1   filename       2231 non-null   object
 2   uniform_type   2231 non-null   object
 3   xmin           2231 non-null   int64 
 4   xmax           2231 non-null   int64 
 5   ymin           2231 non-null   int64 
 6   ymax           2231 non-null   int64 
dtypes: int64(4), object(3)
memory usage: 122.1+ KB


In [5]:
data['uniform_type'].unique()

array(['lawyer', 'Traffic police', 'military force', 'ncc force',
       'commando force', 'navy force', 'air force', 'plumber', 'doctor',
       'student', 'security guard', 'construction worker'], dtype=object)

In [6]:
data['uniform_type'].value_counts()

student                464
construction worker    433
military force         281
Traffic police         258
doctor                 221
lawyer                 205
plumber                126
security guard         121
commando force          60
ncc force               40
air force               16
navy force               6
Name: uniform_type, dtype: int64

In [7]:
img_width = 640
img_height = 480

def width(df):
    return int(df.xmax - df.xmin)
def height(df):
    return int(df.ymax - df.ymin)
def x_center(df):
    return int(df.xmin + (df.width/2))
def y_center(df):
    return int(df.ymin + (df.height/2))
def w_norm(df):
    return df/img_width
def h_norm(df):
    return df/img_height

df = pd.read_csv(r'C:\Users\saikr\object_detection_p2\uniform_data1_detection.csv')

le = preprocessing.LabelEncoder()
le.fit(df['uniform_type'])
print(le.classes_)
labels = le.transform(df['uniform_type'])
df['labels'] = labels

df['width'] = df.apply(width, axis=1)
df['height'] = df.apply(height, axis=1)

df['x_center'] = df.apply(x_center, axis=1)
df['y_center'] = df.apply(y_center, axis=1)

df['x_center_norm'] = df['x_center'].apply(w_norm)
df['width_norm'] = df['width'].apply(w_norm)

df['y_center_norm'] = df['y_center'].apply(h_norm)
df['height_norm'] = df['height'].apply(h_norm)

df.head(30)

['Traffic police' 'air force' 'commando force' 'construction worker'
 'doctor' 'lawyer' 'military force' 'navy force' 'ncc force' 'plumber'
 'security guard' 'student']


,prev_filename,filename,uniform_type,xmin,xmax,ymin,ymax,labels,width,height,x_center,y_center,x_center_norm,width_norm,y_center_norm,height_norm
0,OMG-Oh-My-God-court-room-drama-e1474114545186.jpg,0.jpg,lawyer,26,221,108,387,5,195,279,123,247,0.192188,0.304688,0.514583,0.581250
1,OMG-Oh-My-God-court-room-drama-e1474114545186.jpg,0.jpg,lawyer,456,700,107,319,5,244,212,578,213,0.903125,0.381250,0.443750,0.441667
2,Telangana-Traffic-police.jpg,1.jpg,Traffic police,422,654,177,800,0,232,623,538,488,0.840625,0.362500,1.016667,1.297917
3,Telangana-Traffic-police.jpg,1.jpg,Traffic police,754,956,147,461,0,202,314,855,304,1.335938,0.315625,0.633333,0.654167
4,ap1.jpg,2.jpg,military force,199,325,112,394,6,126,282,262,253,0.409375,0.196875,0.527083,0.587500
5,ap10.jpg,3.jpg,military force,15,272,211,336,6,257,125,143,273,0.223438,0.401562,0.568750,0.260417
6,ap10.jpg,3.jpg,military force,397,612,215,332,6,215,117,504,273,0.787500,0.335938,0.568750,0.243750
7,ap100.jpg,4.jpg,military force,81,173,179,408,6,92,229,127,293,0.198437,0.143750,0.610417,0.477083
8,ap100.jpg,4.jpg,military force,395,476,220,350,6,81,130,435,285,0.679688,0.126562,0.593750,0.270833
9,ap101.jpg,5.jpg,military force,12,92,80,303,6,80,223,52,191,0.081250,0.125000,0.397917,0.464583


In [8]:
df.shape

(2231, 16)

## SPLITING THE DATA INTO TRAIN AND VALID

In [10]:
df_train, df_valid = model_selection.train_test_split(df, test_size=0.2, random_state=13, shuffle=True)
print(df_train.shape, df_valid.shape)

os.mkdir(r'C:\Users\saikr\object_detection_p2\object_split')
os.mkdir(r'C:\Users\saikr\object_detection_p2\object_split\images')
os.mkdir(r'C:\Users\saikr\object_detection_p2\object_split\images\train')
os.mkdir(r'C:\Users\saikr\object_detection_p2\object_split\images\valid')

os.mkdir(r'C:\Users\saikr\object_detection_p2\object_split\labels')
os.mkdir(r'C:\Users\saikr\object_detection_p2\object_split\labels\train')
os.mkdir(r'C:\Users\saikr\object_detection_p2\object_split\labels\valid')

(1784, 16) (447, 16)


In [11]:
def segregate_data(df, img_path, label_path, train_img_path, train_label_path):
    filenames = []
    for filename in df.filename:
        filenames.append(filename)
    filenames = set(filenames)
    
    
    for filename in filenames:
        yolo_list = []
        

        for _,row in df[df.filename == filename].iterrows():
            yolo_list.append([row.labels, row.x_center_norm, row.y_center_norm, row.width_norm, row.height_norm])
            
            
        yolo_list = np.array(yolo_list)
        txt_filename = os.path.join(train_label_path,str(row.prev_filename.split('.')[0])+".txt")
    # Save the .img & .txt files to the corresponding train and validation folders
        np.savetxt(txt_filename, yolo_list, fmt=["%d", "%f", "%f", "%f", "%f"])
        shutil.copyfile(os.path.join(img_path,row.prev_filename), os.path.join(train_img_path,row.prev_filename))

In [12]:
## Apply function ## 
src_img_path = r"C:\Users\saikr\object_detection_p2\obj_dataset\Images"
src_label_path = r"C:\Users\saikr\object_detection_p2\obj_dataset\Annotations"

train_img_path = r"C:\Users\saikr\object_detection_p2\object_split\images\train"
train_label_path = r"C:\Users\saikr\object_detection_p2\object_split\labels\train"

valid_img_path = r"C:\Users\saikr\object_detection_p2\object_split\images\valid"
valid_label_path = r"C:\Users\saikr\object_detection_p2\object_split\labels\valid"

segregate_data(df_train, src_img_path, src_label_path, train_img_path, train_label_path)
segregate_data(df_valid, src_img_path, src_label_path, valid_img_path, valid_label_path)

print("No. of Training images", len(os.listdir(r'C:\Users\saikr\object_detection_p2\object_split\images\train')))
print("No. of Training labels", len(os.listdir(r'C:\Users\saikr\object_detection_p2\object_split\labels\train')))

print("No. of valid images", len(os.listdir(r'C:\Users\saikr\object_detection_p2\object_split\images\valid')))
print("No. of valid labels", len(os.listdir(r'C:\Users\saikr\object_detection_p2\object_split\labels\valid')))

No. of Training images 1224
No. of Training labels 1224
No. of valid images 400
No. of valid labels 400
